In [1]:
import os
import sys
import pandas as pd
import zeep
from zeep import Client
from lxml import etree
from zeep import Plugin

# proxy for testing in rki.local
# os.environ['https_proxy'] = "http://fw-bln.rki.ivbb.bund.de:8020"

# Global
language = "German"
cube = "SurvStat"


# Logging Plugin
class LoggingPlugin(Plugin):
    # Set Printing
    printGeneratedXml = True

    def ingress(self, envelope, http_headers, operation):
        if self.printGeneratedXml:
            print(
                etree.tostring(
                    envelope, pretty_print=True, encoding=sys.stdout.encoding))
        return envelope, http_headers

    def egress(self, envelope, http_headers, operation, binding_options):
        if self.printGeneratedXml:
            print(
                etree.tostring(
                    envelope, pretty_print=True, encoding=sys.stdout.encoding))
        return envelope, http_headers


#create the client object
client = Client(
    'https://tools.rki.de/SurvStat/SurvStatWebService.svc?singleWsdl',
    plugins=[])
# create a type_factory which includes the required data types
factory = client.type_factory('ns2')


# function to get all dimensions from the web Service
# Parameter:
#   cube = "SurvStat"
#   language ="German" || "English"
def GetAllDimensions(cube, language):
    dcRequest = factory.DimensionCollectionRequest(
        Cube=cube, Language=language)
    response = client.service.GetAllDimensions(dcRequest)
    return response


# function to get all hierarchy members from the web Service
# Parameter:
#   cube = "SurvStat"
#   hierarchyId = ~
#   language ="German" || "English"
def GetAllHierarchyMembers(cube, hierarchyId, language):
    hmcRequest = factory.HierarchyMemberCollectionRequest(
        Cube=cube, HierarchyId=hierarchyId, Language=language)
    response = client.service.GetAllHierarchyMembers(hmcRequest)
    return response


# function to get the olap result data from the web Service
# Parameter:
#   ColumnHierarchy: string,
#   Cube: string,
#   HierarchyFilters: ns2:FilterCollection,
#   IncludeNullColumns: bool,
#   IncludeNullRows: bool,
#   IncludeTotalColumn: bool,
#   IncludeTotalRow: bool,
#   Language: string,
#   Measure: string,
#   RowHierarchy: string
def GetOlapResultData(colHierarchy_, cube_, hierarchyFilter_, incNullCols_,
                      incNullRows_, incTotalCol_, incTotalRow_, language_,
                      measure_, rowHierarchy_):
    ordRequest = factory.OlapResultDataRequest(
        ColumnHierarchy=colHierarchy_,
        Cube=cube_,
        HierarchyFilters=hierarchyFilter_,
        IncludeNullColumns=incNullCols_,
        IncludeNullRows=incNullRows_,
        IncludeTotalColumn=incTotalCol_,
        IncludeTotalRow=incTotalRow_,
        Language=language_,
        Measure=measure_,
        RowHierarchy=rowHierarchy_)

    response = client.service.GetOlapResultData(ordRequest)
    return response


colHierarchy = "[PathogenMitKrankheit].[Meldeweg Web71].[Krankheit ID Web71]"

incNullCols = False
incNullRows = False
incTotalCol = False
incTotalRow = False
measure = "[Measures].[FallCount_71]"
rowHierarchy = "[ReportingDate].[YearWeek].[YearWeek]"
resOlapResultData = GetOlapResultData(colHierarchy, cube, None, incNullCols,
                                      incNullRows, incTotalCol, incTotalRow,
                                      language, measure, rowHierarchy)

In [2]:
resOlapResultData.Columns

{
    'QueryResultColumn': [
        {
            'Caption': None,
            'ColumnName': '[PathogenMitKrankheit].[Meldeweg Web71].[Meldeweg ID Web71].&[0].&[0]'
        },
        {
            'Caption': 'Acinetobacter-Infektion oder –Kolonisation',
            'ColumnName': '[PathogenMitKrankheit].[Meldeweg Web71].[Meldeweg ID Web71].&[1].&[198]'
        },
        {
            'Caption': 'Adenovirus-K(eratok)onjunktivitis',
            'ColumnName': '[PathogenMitKrankheit].[Meldeweg Web71].[Meldeweg ID Web71].&[1].&[101]'
        },
        {
            'Caption': 'Arbovirus-Erkrankung',
            'ColumnName': '[PathogenMitKrankheit].[Meldeweg Web71].[Meldeweg ID Web71].&[1].&[201]'
        },
        {
            'Caption': 'Botulismus',
            'ColumnName': '[PathogenMitKrankheit].[Meldeweg Web71].[Meldeweg ID Web71].&[1].&[110]'
        },
        {
            'Caption': 'Brucellose',
            'ColumnName': '[PathogenMitKrankheit].[Meldeweg Web71].[Meldeweg ID

In [47]:
columns = []
for column_obj in resOlapResultData.Columns.QueryResultColumn:
    caption = column_obj.Caption
    if caption is not None: 
        columns.append(caption)
columns = ['Kalenderwoche'] + columns
columns

['Kalenderwoche',
 'Acinetobacter-Infektion oder –Kolonisation',
 'Adenovirus-K(eratok)onjunktivitis',
 'Arbovirus-Erkrankung',
 'Botulismus',
 'Brucellose',
 'Campylobacter-Enteritis',
 'Chikungunya',
 'Cholera',
 'CJK (Creutzfeldt-Jakob-Krankheit)',
 'Clostridium difficile',
 'Denguefieber',
 'Diphtherie',
 'E.-coli-Enteritis',
 'Ebolafieber',
 'EHEC-Erkrankung',
 'Enterobacteriaceae-Infektion oder –Kolonisation',
 'Fleckfieber',
 'FSME (Frühsommer-Meningoenzephalitis)',
 'Giardiasis',
 'Haemophilus influenzae, invasive Erkrankung',
 'Hantavirus-Erkrankung',
 'Hepatitis A',
 'Hepatitis B',
 'Hepatitis C',
 'Hepatitis D',
 'Hepatitis E',
 'Hepatitis Non A-E',
 'HUS (Hämolytisch-urämisches Syndrom), enteropathisch',
 'Influenza',
 'Keuchhusten',
 'Kryptosporidiose',
 'Lassafieber',
 'Läuserückfallfieber',
 'Legionellose',
 'Lepra',
 'Leptospirose',
 'Listeriose',
 'Masern',
 'Meningokokken, invasive Erkrankung',
 'Milzbrand',
 'MRSA, invasive Infektion',
 'Mumps',
 'Norovirus-Gastroent

In [67]:
rows = []
for query_row in resOlapResultData.QueryResults.QueryResultRow:
    row = []
    row.append(query_row.Caption)
    for i, val in enumerate(query_row.Values.string):
        if i == 0:
            continue
        row_val = int(val.replace('.', '')) if val is not None else 0
        row.append(row_val)
    rows.append(row)

In [68]:
weekly_diseases = pd.DataFrame.from_records(data=rows, columns=columns)
weekly_diseases

,Kalenderwoche,Acinetobacter-Infektion oder –Kolonisation,Adenovirus-K(eratok)onjunktivitis,Arbovirus-Erkrankung,Botulismus,Brucellose,Campylobacter-Enteritis,Chikungunya,Cholera,CJK (Creutzfeldt-Jakob-Krankheit),...,"Meningitis, andere",Mycoplasma,Parainfluenza,"Pneumokokken, invasive Erkrankung",Ringelröteln,RSV-Infektion,Scharlach,Tetanus,Tollwutexpositionsverdacht,Toxoplasmose
0,2001-KW01,0,0,0,0,0,427,0,0,3,...,1,0,0,0,0,0,0,0,0,0
1,2001-KW02,0,0,0,0,0,823,0,0,3,...,0,0,0,0,0,0,1,0,0,0
2,2001-KW03,0,1,0,0,0,1052,0,0,4,...,0,0,0,0,0,0,0,0,0,0
3,2001-KW04,0,0,0,0,0,885,0,0,2,...,0,0,0,0,0,0,0,0,0,0
4,2001-KW05,0,0,0,0,1,779,0,0,3,...,0,0,0,1,0,0,0,0,0,0
5,2001-KW06,0,2,0,0,1,806,0,0,4,...,1,0,0,0,0,0,0,0,0,0
6,2001-KW07,0,3,0,0,0,819,0,0,0,...,0,0,0,1,0,0,0,0,0,0
7,2001-KW08,0,2,0,0,1,752,0,0,1,...,0,0,0,1,0,0,1,0,0,0
8,2001-KW09,0,3,0,2,1,801,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,2001-KW10,0,4,0,0,0,709,0,0,3,...,1,0,0,0,0,0,0,0,0,0


In [70]:
weekly_diseases.to_pickle('weekly_diseases.pkl')